In [2]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [17]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    keys_list = []
    for _ in range(n):
        x_temp, y_temp = generate_keys(g, p)
        keys_list.append([x_temp, y_temp])
    return keys_list


def concat_and_shuffle_total_keys(keys_list, x, y):
    keys_list.append([x, y])
    random.shuffle(keys_list)
    return keys_list


def get_real_key_position(keys_list, y):
    public_key_list = [key[1] for key in keys_list]
    return public_key_list.index(y)


def get_Y(keys_list):
    public_key_list = [key[1] for key in keys_list]
    Y = ''.join([str(i) for i in public_key_list])
    return Y


def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R


def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    e_list = [None for _ in range(len(keys_list))]
    sig_list = [None for _ in range(len(keys_list))]
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    y_wave = pow(h, x_real, p)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    e_list[idx] = e_initial
    while keys_list[idx][1] != y_real:
        e_last_one = e_list[idx]
        _, random_sig = generate_keys(g, p)
        sig_list[idx] = random_sig
        temp_y = keys_list[idx][1]
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        idx = (idx + 1) % len(total_keys_list)
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
        'e_list': e_list
    }
    return total_res
        
    
    

# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 100
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
print(total_keys_list)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



[[928823193, 384685004], [904063068, 568853885], [669505685, 793576158], [67403236, 433670455], [843648359, 90356996], [975592561, 819260894], [844434796, 244171510], [159428138, 301074624], [398589287, 434115538], [284502510, 482520269], [182396094, 479869817], [191445921, 36417491], [83957432, 481067266], [105001241, 120735045], [295294525, 807573010], [717953762, 688857664], [41195402, 441935263], [59368580, 791548401], [334763868, 147923054], [576055607, 603051737], [243775361, 160833348], [850022724, 691133998], [139457666, 838592433], [135753701, 775757146], [519446152, 294352916], [401475249, 364724750], [898924568, 645644496], [655552587, 708643236], [43912819, 372423527], [976139712, 628112890], [483228234, 254742686], [373190040, 870812049], [136101791, 629674849], [883030831, 568159170], [363741301, 824685017], [937019625, 149008698], [8805590, 706141269], [952049322, 983023578], [61877807, 814674630], [374981032, 286502466], [896461778, 104169092], [988354578, 260631003], [

{'e1': 16492908317768436330327981066096012980923803650382738768935332158843557023864,
 'sig_list': [360783005,
  523010985,
  349536160,
  441798260,
  595459548,
  671389892,
  804627093,
  245824790,
  41143921,
  648419676,
  687271990,
  126269338,
  787246483,
  729689024,
  71834774,
  743303060,
  124191536,
  367120217,
  911411469,
  653045752,
  854332300,
  893530107,
  491108481,
  40263638,
  754068644,
  241896527,
  339788939,
  447642853,
  819851876,
  928662371,
  699425869,
  36095045,
  527370584,
  128417064,
  119896091,
  549891688,
  546974785,
  573230415,
  460063411,
  56446486,
  771146972,
  884240944,
  905699211,
  903457735,
  834651363,
  942764900,
  790054910,
  877605719,
  763837177,
  205777293,
  948736091,
  482016377,
  346263279,
  298239315,
  182503168,
  10642419,
  355621464,
  138057970,
  10686672,
  65712471,
  206941472,
  149342060,
  834448983,
  615950647,
  152049898,
  251319923,
  934289883,
  356463819,
  876482656,
  176231393,


In [18]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 16492908317768436330327981066096012980923803650382738768935332158843557023864
temp_e_V: 9333217433707507073620679624618927618405686042588172056085277881807557209011
temp_e_V: 100670763915848226080708762414939119689632077352452238190112097870700390188216
temp_e_V: 48897734292085635773522532585802941113851871715794751077789356565411460721630
temp_e_V: 103175432665186445367746549538727386045980376477349815419146328945660231489094
temp_e_V: 83290127832453351358913687533968558010720031979721953157458669605320714179360
temp_e_V: 68718420801954944749775043660570671574003750003454053367602206608812549902873
temp_e_V: 10334188119680196282102401659109696718579215252671709274011507535326086412613
temp_e_V: 20920148484540088857784694989594634359200379275209639388534532954541300529026
temp_e_V: 35853950962049736745815684567987575324369132276605560400701503637556357785877
temp_e_V: 69975176510482423066558300586835707582663536773241310389451239023224245156715
temp_e_V: 730837905199367977503

In [16]:
msg_1 = 'new test'

total_res = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
total_res

s_real: 172199989


{'e1': 4163613595304821537172991371039638639289302562590240052484297960667891800521,
 'sig_list': [501239345,
  87751165,
  215720556,
  228090841,
  941100005,
  829576975,
  315473474,
  46280039,
  359970405,
  611733846,
  908987074,
  657882151,
  646232390,
  385855154,
  417011530,
  197718719,
  232455944,
  271110586,
  767504614,
  28642248,
  444873734,
  304533619,
  104888450,
  232888276,
  804934823,
  532192699,
  513983074,
  795554257,
  675499684,
  561753284,
  702490194,
  640662958,
  282066166,
  102221075,
  574161153,
  912138156,
  11845817,
  433353737,
  908885570,
  614264276,
  216598559,
  23796792,
  388509247,
  464729450,
  362891075,
  436685827,
  177401809,
  53821865,
  56904039,
  784662194,
  786303088,
  775554838,
  172199989,
  570248248,
  259371779,
  964847856,
  144988066,
  692633314,
  39338051,
  698174359,
  354358622,
  285347648,
  416995663,
  4325153,
  204469618,
  497822550,
  275843994,
  169923874,
  926001492,
  766999234,
  2

In [20]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True
